In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
conll = datasets.load_dataset("conll2003")
CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names
print(CONLL_NER_TAGS)
conll["test"][2]

Found cached dataset conll2003 (/Users/kirillsergeev/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 484.26it/s]

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


{'id': '2',
 'tokens': ['AL-AIN', ',', 'United', 'Arab', 'Emirates', '1996-12-06'],
 'pos_tags': [22, 6, 22, 22, 23, 11],
 'chunk_tags': [11, 0, 11, 12, 12, 12],
 'ner_tags': [5, 0, 5, 6, 6, 0]}

In [3]:
from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

# Load pretrained model and tokenizer for English NER task (dslim/bert-base-NER)
model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

In [4]:
import numpy as np
import torch
from tqdm import tqdm

class ModelWrapper():
    def __init__(self, model, tokenizer):
        self.model = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy=None)

    def _unite_entities(self, entities):
        if len(entities) <= 1:
            return entities

        united_result = []
        cur_entity = {key: entities[0][key] for key in ['entity', 'word', 'start', 'end']}
        for entity in entities[1:]:
            if entity['word'].startswith('##'):
                cur_entity['word'] += entity['word'].lstrip('#')
                cur_entity['end'] = entity['end']
            else:
                united_result.append(cur_entity)
                cur_entity = {key: entity[key] for key in ['entity', 'word', 'start', 'end']}
        united_result.append(cur_entity)
        return united_result

    def _convert_entities_to_bio(self, tokens, entities):
        bio_tags = []
        cur_entity_idx = 0
        for token in tokens:
            if (cur_entity_idx < len(entities))\
                    and (token == entities[cur_entity_idx]['word']):
                bio_tags.append(entities[cur_entity_idx]['entity'])
                cur_entity_idx += 1
            else:
                bio_tags.append('O')
        return bio_tags

    def _postprocessing(self, tokens, model_output):
        entities = self._unite_entities(model_output)
        bio_tags = self._convert_entities_to_bio(tokens, entities)
        return bio_tags

    def predict(self, X):
        with torch.no_grad():
            ner_entitites = self.model(X)
            tags = []
            for text, entities in tqdm(zip(X, ner_entitites)):
                tags.append(self._postprocessing(text.split(), entities))
            return tags

wrapped_model = ModelWrapper(model, tokenizer)

In [5]:
def preprocessing_dataset(dataset):
    result = {'X': [], 'y_true': []}
    for sample in dataset:
        result['X'].append(' '.join(sample['tokens']))
        result['y_true'].append([CONLL_NER_TAGS[tag] for tag in sample['ner_tags']])
    return result

In [6]:
# from itertools import islice
# train_data = preprocessing_dataset(conll['train'])
# train_data['y_pred'] = wrapped_model.predict(train_data['X'])

# oos_data = preprocessing_dataset(conll['test'])
# oos_data['y_pred'] = wrapped_model.predict(oos_data['X'])

In [9]:
# import pickle

# with open('./train_data.pkl', 'wb') as f:
#     pickle.dump(train_data, f)
# with open('./oos_data.pkl', 'wb') as f:
#     pickle.dump(oos_data, f)

In [9]:
import pickle

with open('./train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('./oos_data.pkl', 'rb') as f:
    oos_data = pickle.load(f)

In [16]:
print(all([len(i) == len(j) for i, j in zip(train_data['y_true'], train_data['y_pred'])]))
print(all([len(i) == len(j) for i, j in zip(oos_data['y_true'], oos_data['y_pred'])]))

True
True


In [18]:
from sbe_vallib.validation.sampler.ner_sampler import NerSampler

sampler = NerSampler(train=train_data, oos=oos_data)
sampler.set_seed(42, bootstrap=True)

In [19]:
list(set([tag.split('-')[-1] for tag in CONLL_NER_TAGS]) - set(['O']))

['PER', 'MISC', 'ORG', 'LOC']

In [21]:
from sbe_vallib.validation.scorer.ner_scorer import NerScorer
from sbe_vallib.validation.utils.metrics import NER_IOB_METRICS

scorer = NerScorer(metrics=NER_IOB_METRICS, is_calc_ner_metrics=False, tags=['PER', 'MISC', 'LOC', 'ORG'])
scores = scorer.score(oos_data['y_true'], oos_data['y_pred'])

In [24]:
from sbe_vallib import BaseValidation
validor = BaseValidation(model, sampler, scorer, pipeline="sbe_vallib/validation/nlp/pipelines/Config_45.xlsx")
validor.pipeline['tests']

{'train_test_independence_test': {'import_path': 'sbe_vallib.validation.nlp.general_tests.data_quality.test_train_test_independence.train_test_independence_test',
  'block_key': 'data_quality',
  'params': {},
  'informative': 1},
 'test_extremal_missing_values': {'import_path': 'sbe_vallib.validation.nlp.general_tests.model_quality.test_key_metric.test_key_metric',
  'block_key': 'data_quality',
  'params': {'n_iter': 200},
  'informative': 0},
 'psi_factor_classes_test_oos': {'import_path': 'sbe_vallib.validation.nlp.general_tests.data_quality.test_train_test_independence.train_test_independence_test',
  'block_key': 'data_quality',
  'params': {},
  'informative': 1},
 'psi_factor_classes_test_oot': {'import_path': 'sbe_vallib.validation.nlp.general_tests.model_quality.test_key_metric.test_key_metric',
  'block_key': 'data_quality',
  'params': {'n_iter': 200},
  'informative': 0},
 'test_key_metric': {'import_path': 'sbe_vallib.validation.nlp.general_tests.data_quality.test_train_t

In [27]:
res = validor.validate()

	Train/oos samples are too huge for independence test, calculating on 0.4287 fraction
(7499, 85)


100%|██████████| 20/20 [00:01<00:00, 12.45it/s]


	Train/oos samples are too huge for independence test, calculating on 0.4287 fraction
(7499, 85)


100%|██████████| 20/20 [00:01<00:00, 12.88it/s]


	Train/oos samples are too huge for independence test, calculating on 0.4287 fraction
(7499, 85)


...

100%|██████████| 20/20 [00:01<00:00, 12.75it/s]


	Train/oos samples are too huge for independence test, calculating on 0.4287 fraction
(7499, 85)


100%|██████████| 20/20 [00:01<00:00, 12.57it/s]


	Train/oos samples are too huge for independence test, calculating on 0.4287 fraction
(7499, 85)


100%|██████████| 20/20 [00:01<00:00, 13.28it/s]


In [28]:
res.keys()

dict_keys(['train_test_independence_test', 'test_extremal_missing_values', 'psi_factor_classes_test_oos', 'psi_factor_classes_test_oot', 'test_key_metric', 'test_confidence_inteval', 'test_presicion', 'test_recall', 'test_2_5', 'test_3_1', 'test_3_2', 'test_feature_importance', 'test_4_2', 'test_key_metric_stability', 'test_presicion_stability', 'test_recall_stability', 'test_5_4', 'test_5_6', 'test_5_7'])

In [30]:
res['test_presicion'], res['test_2_5']

({'semaphore': 'grey',
  'result_dict': {'result': {'Фактический gini': 0.18245832161197367,
    'Квантиль': 1,
    'Результат теста': 'Grey'}},
  'result_dataframes': [   Фактический gini  Квантиль Результат теста
   0          0.182458         1            Grey],
  'result_plots': []},
 {'semaphore': 'grey',
  'result_dict': {'result': {'Фактический gini': 0.18245832161197367,
    'Квантиль': 1,
    'Результат теста': 'Grey'}},
  'result_dataframes': [   Фактический gini  Квантиль Результат теста
   0          0.182458         1            Grey],
  'result_plots': []})

In [23]:
scorer.ner_metrics(oos_data['y_true'], oos_data['y_pred'])

{'ent_type_f1': 0.8238541153277475,
 'ent_type_f1_by_PER': 0.8030194104960461,
 'ent_type_f1_by_MISC': 0.7612179487179488,
 'ent_type_f1_by_LOC': 0.8062648961525366,
 'ent_type_f1_by_ORG': 0.8829452485840151,
 'ent_type_recall': 0.7399079320113314,
 'ent_type_recall_by_PER': 0.6907854050711194,
 'ent_type_recall_by_MISC': 0.6766381766381766,
 'ent_type_recall_by_LOC': 0.709832134292566,
 'ent_type_recall_by_ORG': 0.8446718844069837,
 'ent_type_precision': 0.9292861907938625,
 'ent_type_precision_by_PER': 0.9587982832618026,
 'ent_type_precision_by_MISC': 0.86996336996337,
 'ent_type_precision_by_LOC': 0.9330181245074862,
 'ent_type_precision_by_ORG': 0.924851680949242,
 'partial_f1': 0.8498767865943814,
 'partial_f1_by_PER': 0.8076923076923076,
 'partial_f1_by_MISC': 0.7908653846153845,
 'partial_f1_by_LOC': 0.8478038815117467,
 'partial_f1_by_ORG': 0.9118942731277533,
 'partial_recall': 0.7632790368271954,
 'partial_recall_by_PER': 0.6948051948051948,
 'partial_recall_by_MISC': 0.7029